In [1]:
from azure.identity import DeviceCodeCredential,ClientSecretCredential
import azureHelper
import pandas as pd

In [ ]:
tenant_id = '<TENANT ID>'
client_id = '<CLIENT ID>'
client_secret = '<SECRET>'
roleThreshold = 3

credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret)
credential2 = DeviceCodeCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret)

graphToken = credential.get_token("https://graph.microsoft.com/.default",scope=["Application.Read.All","User.Read.All"])
azureToken = credential2.get_token("https://management.azure.com/user_impersonation")
# The app must be registered under tenant to authenticate

In [ ]:
users = azureHelper.getUsers(graphToken)
mfaStatus = azureHelper.getMFAStatus(graphToken)
subscriptions = azureHelper.getSubscriptions(azureToken)
resources = azureHelper.getResources(azureToken,[x[0] for x in subscriptions])
apps = azureHelper.getApps(graphToken)
resourcesDF = pd.DataFrame(resources, columns=["id","name","type","properties"])
sqlServers = resourcesDF[resourcesDF.type == "microsoft.sql/servers"]
virtualmachines = resourcesDF[resourcesDF.type == "microsoft.compute/virtualmachines"]
storageAccounts = resourcesDF[resourcesDF.type ==  "microsoft.storage/storageaccounts"]
appServices = resourcesDF[resourcesDF.type == "microsoft.web/sites"]
keyVaults = resourcesDF[resourcesDF.type ==  "microsoft.keyvault/vaults"]
managedIdentities = resourcesDF[resourcesDF.type ==  "microsoft.managedidentity/userassignedidentities"]
logicApps = resourcesDF[resourcesDF.type ==  "microsoft.logic/workflows"]
vmDisks = resourcesDF[resourcesDF.type ==  "microsoft.compute/disks"]
networkSecurityGroups = resourcesDF[resourcesDF.type == "microsoft.network/networksecuritygroups"]

In [ ]:
auditResults = []
# sampleEntry={
#     "name":"Issue Name",
#     "details":"",
#     "issueSeverity":"High",
#     "affectedResource":"",
#     "reference":"",
#     "remediationInstructions":""
# }
## Identity Checks
for user in mfaStatus.keys():
    if mfaStatus[user] == False:
        issue = {
            "name":"MFA not enabled for user",
            "details":"Multi factor authentication is an essential security measure against Phishing and other identity attacks. Need to enable the same.",
            "issueSeverity":"High",
            "affectedResource":user,
            "category":"Identity-User",
            "reference":"https://www.zoho.com/blog/directory/why-is-mfa-important-for-your-business.html#:~:text=Eighty%2Done%20percent%20of%20hacking,also%20works%20well%20with%20SSO.",
            "remediationInstructions":"https://learn.microsoft.com/en-us/entra/identity/authentication/tutorial-enable-azure-mfa"
        }
        auditResults.append(issue)


## Checking Apps

for app in apps:
    ## checking for password credentials
    if len(app["passwordCredentials"]) > 0:
        issue = {
            "name":"Password credentials present for an application",
            "details":"",
            "issueSeverity":"Medium",
            "affectedResource":"AppId:"+ app['appId'],
            "category":"Identity-Apps",
            "reference":"",
            "remediationInstructions":""
        }
        auditResults.append(issue)
    ## checking for dangling reply URLs
    replyUrls = app['publicClient']['redirectUris'] + app['web']['redirectUris'] + app['spa']['redirectUris']
    for uri in replyUrls:
        if azureHelper.checkLive(uri) == False:
            issue = {
                "name":"Reply URI is dangling",
                "details":"",
                "issueSeverity":"Medium",
                "affectedResource":"AppId:"+ app['appId'] + "; Dangling URI:" + uri,
                "category":"Identity-Apps",
                "reference":"",
                "remediationInstructions":""
            }
            auditResults.append(issue)

## RBAC count based check

roles = []
roleAgg = {}
for res in resources:
    result = azureHelper.getRoleAssignments(res,azureToken)
    if len(result) > 0:
        roles = roles + ([x['properties'] for x in result])
for role in roles:
    if role["principalId"] not in roleAgg.keys():
        roleAgg[role['principalId']] = 0
    else:
        roleAgg[role['principalId']] = roleAgg[role['principalId']] + 1

for i in roleAgg.keys():
    if roleAgg[i] > roleThreshold:
        issue = {
                "name":"Over Privileged User",
                "details":"A user with too many roles",
                "issueSeverity":"Medium",
                "affectedResource":"UserId:"+ i,
                "category":"Identity-RBAC",
                "reference":"",
                "remediationInstructions":""
            }
        auditResults.append(issue)
    
## Network Security Groups evaluation

secRules = []
for group in networkSecurityGroups.id.values:
    secRules = secRules + azureHelper.getSecurityGroupRules(group, azureToken)
for rule in secRules:
    if rule['properties']['direction'] == "Inbound" and rule['properties']['destinationPortRange'] in ["22","3389","21"] and rule['properties']['sourcePortRange'] in ["*","0.0.0.0"]:
        issue = {
                "name":"Ports used for remote control/access are open to the Internet",
                "details":"",
                "issueSeverity":"High",
                "affectedResource": rule['id'],
                "category":"Network",
                "reference":"",
                "remediationInstructions":""
            }
        auditResults.append(issue)

## VM Disks Access
for i in vmDisks.id.values:
    res = azureHelper.getDiskInfo(i, azureToken)
    if res['networkAccessPolicy'] == "AllowAll":
        issue = {
                "name":"VM Disk accessible to public networks.",
                "details":"",
                "issueSeverity":"High",
                "affectedResource": i,
                "category":"Network",
                "reference":"",
                "remediationInstructions":""
            }
        auditResults.append(issue)
        
## All resources audit settings
for i in resourcesDF.id.values:
    res = azureHelper.getAuditSettings(i, azureToken)
    # print(res)
    if len(res) > 0:
        for x in res:
            for j in x['properties']['logs']:
                if j['enabled'] == False or j['retentionPolicy']['enabled'] == False:
                    issue = {
                        "name":"Audit Settings not enabled or retention policy is not enabled.",
                        "details":"Audit settings not enabled. This means you don't have visibility into the activity that's happ. You can't protect what you can't see.",
                        "issueSeverity":"High",
                        "affectedResource": i,
                        "category":"Network",
                        "reference":"",
                        "remediationInstructions":"Enable logging through diagnostic settings."
                    }
                    auditResults.append(issue)
